In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import itertools
from scipy.special import kn
from scipy import optimize
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read data

In [4]:
fname = '/Users/theoares/Dropbox (MIT)/research/0nubb/analysis_output/24I/ml0p01/Z_gamma_error.h5'
f = h5py.File(fname, 'r')
Zq = np.real(f['Zq'][()][0])
ZV = np.real(f['ZV'][()][0])
ZA = np.real(f['ZA'][()][0])
Z = np.zeros((len(Zq), 5, 5), dtype = np.float64)
for i, j in itertools.product(range(5), repeat = 2):
    key = 'Z' + str(i + 1) + str(j + 1)
    Z[:, i, j] = np.real(f[key][()])

In [5]:
print('Zq = ' + str(np.mean(Zq)) + ' pm ' + str(np.std(Zq, ddof = 1)))
print('ZV = ' + str(np.mean(ZV)) + ' pm ' + str(np.std(ZV, ddof = 1)))
print('ZA = ' + str(np.mean(ZA)) + ' pm ' + str(np.std(ZA, ddof = 1)))
print('Z_ij = ' + str(np.mean(Z, axis = 0)) + ' pm ' + str(np.std(Z, ddof = 1, axis = 0)))

Zq = 1.127558 pm 0.028953278
ZV = 0.71805817 pm 0.014095646
ZA = 0.79719186 pm 0.017124178
Z_ij = [[ 9.64706164e-01 -2.18588998e-01 -1.23463846e-01  7.20640275e-02
   3.98792690e-02]
 [-1.88279639e-01  1.01624295e+00  5.96461819e-01 -8.07477519e-02
  -2.55688741e-02]
 [ 1.21247992e-02  5.16378818e-02  2.53779943e-01  2.73579595e-01
  -1.96878253e-02]
 [ 3.49224188e-02  1.84832388e-02  1.62866369e-01  3.80574795e-01
  -1.93776556e-02]
 [ 3.72076840e-02  1.56135105e-03  4.35220132e-02 -7.67586708e-01
   1.73294319e+00]] pm [[0.04524187 0.01996437 0.01160432 0.01030337 0.00883432]
 [0.018528   0.05257612 0.03268581 0.01309977 0.01472278]
 [0.01145325 0.02061155 0.02944939 0.04600564 0.03116078]
 [0.01292803 0.01580144 0.02486413 0.04789667 0.0329349 ]
 [0.00845991 0.00824655 0.00750805 0.04193483 0.08384164]]


In [6]:
# Extract components of Z_{ij} with operator mixing
# Running effects seem to be much larger for the last 3 components than for the first 2, which could explain why the 
# first few are much closer to 1. I'm interested to see what happens when I run these to the matching point and 
# convert to MSbar, I expect they'll be closer to 1. (i.e. see Figures 7 - 10 in the Kaon mixing paper.)
print('(27, 1)')
print('Z_11 = ' + str(np.mean(Z[:, 0, 0])) + ' pm ' + str(np.std(Z[:, 0, 0], ddof = 1)))
print('\n(8, 8)')
print('Z_22 = ' + str(np.mean(Z[:, 1, 1])) + ' pm ' + str(np.std(Z[:, 1, 1], ddof = 1)))
print('Z_23 = ' + str(np.mean(Z[:, 1, 2])) + ' pm ' + str(np.std(Z[:, 1, 2], ddof = 1)))
print('Z_32 = ' + str(np.mean(Z[:, 2, 1])) + ' pm ' + str(np.std(Z[:, 2, 1], ddof = 1)))
print('Z_33 = ' + str(np.mean(Z[:, 2, 2])) + ' pm ' + str(np.std(Z[:, 2, 2], ddof = 1)))
print('\n(6, bar{6})')
print('Z_44 = ' + str(np.mean(Z[:, 3, 3])) + ' pm ' + str(np.std(Z[:, 3, 3], ddof = 1)))
print('Z_45 = ' + str(np.mean(Z[:, 3, 4])) + ' pm ' + str(np.std(Z[:, 3, 4], ddof = 1)))
print('Z_54 = ' + str(np.mean(Z[:, 4, 3])) + ' pm ' + str(np.std(Z[:, 4, 3], ddof = 1)))
print('Z_55 = ' + str(np.mean(Z[:, 4, 4])) + ' pm ' + str(np.std(Z[:, 4, 4], ddof = 1)))

(27, 1)
Z_11 = 0.9647061641399677 pm 0.045241871894796545

(8, 8)
Z_22 = 1.0162429488622224 pm 0.052576122486673454
Z_23 = 0.596461818768428 pm 0.03268580532941133
Z_32 = 0.05163788179365488 pm 0.020611549989190275
Z_33 = 0.25377994317274827 pm 0.029449391502461734

(6, bar{6})
Z_44 = 0.3805747949160062 pm 0.04789666878858301
Z_45 = -0.01937765559145751 pm 0.0329348964004576
Z_54 = -0.7675867080688477 pm 0.041934827816143265
Z_55 = 1.7329431863931508 pm 0.08384164253330548


In [12]:
def get_std_range(data):
    n_samples = len(data)
    sample_range = range(2, len(data))
    errors = np.zeros(len(sample_range), dtype = np.float64)
    for idx, n in enumerate(sample_range):
        subdata = data[:n]
        subdata_boot = bootstrap(subdata, Nb = 10)
        errors[idx] = np.std(subdata_boot, ddof = 1)
    return errors

In [22]:
Zq_std = get_std_range(Zq)
print('Error variation')
print(Zq_std)
print('\nRaw data for Zq')
print(Zq)

Error variation
[0.00426734 0.00655951 0.00477323 0.0103822  0.00732325 0.0074208
 0.01337164 0.01143453 0.00654077 0.00762282 0.00431678]

Raw data for Zq
[1.1463767 1.1571964 1.1320938 1.1418558 1.0829463 1.1042217 1.16459
 1.0936686 1.1187632 1.0975311 1.143033  1.1054981 1.1704788]


In [20]:
Z11_std = get_std_range(Z[:, 0, 0])
print('Error variation')
print(Z11_std)
print('\nRaw data for Z11')
print(Z[:, 0, 0])

Error variation
[0.00987879 0.0193413  0.01554036 0.01714636 0.01229979 0.0144502
 0.02198467 0.02020836 0.01150502 0.01210305 0.00703886]

Raw data for Z11
[1.00449467 1.02954197 0.95184058 0.97307116 0.89899749 0.91992623
 1.03037477 0.91762978 0.94866258 0.92881095 1.01584899 0.93361229
 0.98836869]


In [23]:
Z22_std = get_std_range(Z[:, 1, 1])
print('Error variation')
print(Z22_std)
print('\nRaw data for Z22')
print(Z[:, 1, 1])

Error variation
[0.02191894 0.032085   0.02281209 0.01976816 0.01490004 0.01942518
 0.02438198 0.0253465  0.01624331 0.01129509 0.00902434]

Raw data for Z22
[1.04440701 1.09998167 0.97874707 1.02788234 0.94621605 0.95653564
 1.10393548 0.97646785 1.00751805 0.96715873 1.04393137 0.99386007
 1.06451702]
